In [1]:
# 국토교통부 버스노선(서울, 부산 제외)
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from ServiceKey import public_serviceKey,mySQL_Connect

mydb = mySQL_Connect()

cursor = mydb.cursor(prepared=True)
citycode = '33360'

# DB에서 도시 코드 구해오기
# 2021.09.22 데이터 삽입완료
query = ("select citycode from public_busctycodelist where citycode >= {}".format(citycode))

cursor.execute(query)

location_list = []

for i in cursor:
    location_list.append(i)
    
b_cnt = 0
for j in location_list:
    location = j
    if b_cnt == 1:
        mydb.close()
        break
    
    for i in range(1,121):
        routeNo = i
        print("location : {} | {}번째 조회시작".format(location[0],routeNo))    
        url = 'http://openapi.tago.go.kr/openapi/service/BusRouteInfoInqireService/getRouteNoList'
        queryParams = '?' + urlencode({
            quote_plus('serviceKey') : public_serviceKey(),
            quote_plus('cityCode') : location[0],
            quote_plus('routeNo') : routeNo,
            quote_plus('numOfRows') : '10000'
        })

        response = requests.get(url + queryParams).text.encode('utf-8')
        xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

        rows = xmlobj.findAll('item')
        resultCode = xmlobj.findAll('resultCode')
        if resultCode[0].text == '99':
            print("location : {} | {} 번째부터 조회수 제한이 걸렸습니다.".format(location[0],routeNo))
            b_cnt += 1
            break
        if rows == []:
            print("{} 번쨰데이터가없습니다.".format(routeNo))
            continue
        for j in rows:
            item = j
            routeno = item.findAll('routeno')
            if routeno == []:
                routeno = None
            else:
                routeno = routeno[0].text

            routeid = item.findAll('routeid')
            if routeid == []:
                routeid = None
            else:
                routeid = routeid[0].text

            routetp = item.findAll('routetp')
            if routetp == []:
                routetp = None
            else:
                routetp = routetp[0].text

            startnodenm = item.findAll('startnodenm')
            if startnodenm == []:
                startnodenm = None
            else:
                startnodenm = startnodenm[0].text

            startvehicletime = item.findAll('startvehicletime')
            if startvehicletime == []:
                startvehicletime = None
            else:
                startvehicletime = startvehicletime[0].text

            endnodenm = item.findAll('endnodenm')
            if endnodenm == []:
                endnodenm = None
            else:
                endnodenm = endnodenm[0].text

            endvehicletime = item.findAll('endvehicletime')
            if endvehicletime == []:
                endvehicletime = None
            else:
                endvehicletime = endvehicletime[0].text
                
            sql = "INSERT INTO public_busroute_info (routeno, routeid, routetp, startnodenm, startvehicletime, endnodenm, endvehicletime,citycode) VALUES (?,?,?,?,?,?,?,?)"
            val = (routeno, routeid, routetp, startnodenm, startvehicletime, endnodenm, endvehicletime, location[0])


            cursor.execute(sql, val)
            mydb.commit()

location : 32410 | 1번째 조회시작
location : 32410 | 2번째 조회시작
location : 32410 | 3번째 조회시작
location : 32410 | 4번째 조회시작
4 번쨰데이터가없습니다.
location : 32410 | 5번째 조회시작
location : 32410 | 6번째 조회시작
location : 32410 | 7번째 조회시작
location : 32410 | 8번째 조회시작
location : 32410 | 9번째 조회시작
location : 32410 | 10번째 조회시작
10 번쨰데이터가없습니다.
location : 32410 | 11번째 조회시작
location : 32410 | 12번째 조회시작
location : 32410 | 13번째 조회시작
13 번쨰데이터가없습니다.
location : 32410 | 14번째 조회시작
14 번쨰데이터가없습니다.
location : 32410 | 15번째 조회시작
15 번쨰데이터가없습니다.
location : 32410 | 16번째 조회시작
16 번쨰데이터가없습니다.
location : 32410 | 17번째 조회시작
17 번쨰데이터가없습니다.
location : 32410 | 18번째 조회시작
18 번쨰데이터가없습니다.
location : 32410 | 19번째 조회시작
19 번쨰데이터가없습니다.
location : 32410 | 20번째 조회시작
20 번쨰데이터가없습니다.
location : 32410 | 21번째 조회시작
location : 32410 | 22번째 조회시작
location : 32410 | 23번째 조회시작
23 번쨰데이터가없습니다.
location : 32410 | 24번째 조회시작
24 번쨰데이터가없습니다.
location : 32410 | 25번째 조회시작
25 번쨰데이터가없습니다.
location : 32410 | 26번째 조회시작
26 번쨰데이터가없습니다.
location : 32410 | 27번째 조회시작
27 번쨰데이터가없습니다.
lo

location : 33010 | 92번째 조회시작
location : 33010 | 93번째 조회시작
93 번쨰데이터가없습니다.
location : 33010 | 94번째 조회시작
94 번쨰데이터가없습니다.
location : 33010 | 95번째 조회시작
95 번쨰데이터가없습니다.
location : 33010 | 96번째 조회시작
96 번쨰데이터가없습니다.
location : 33010 | 97번째 조회시작
97 번쨰데이터가없습니다.
location : 33010 | 98번째 조회시작
98 번쨰데이터가없습니다.
location : 33010 | 99번째 조회시작
99 번쨰데이터가없습니다.
location : 33010 | 100번째 조회시작
100 번쨰데이터가없습니다.
location : 33010 | 101번째 조회시작
location : 33010 | 102번째 조회시작
102 번쨰데이터가없습니다.
location : 33010 | 103번째 조회시작
103 번쨰데이터가없습니다.
location : 33010 | 104번째 조회시작
104 번쨰데이터가없습니다.
location : 33010 | 105번째 조회시작
location : 33010 | 106번째 조회시작
106 번쨰데이터가없습니다.
location : 33010 | 107번째 조회시작
107 번쨰데이터가없습니다.
location : 33010 | 108번째 조회시작
108 번쨰데이터가없습니다.
location : 33010 | 109번째 조회시작
109 번쨰데이터가없습니다.
location : 33010 | 110번째 조회시작
110 번쨰데이터가없습니다.
location : 33010 | 111번째 조회시작
location : 33010 | 112번째 조회시작
location : 33010 | 113번째 조회시작
location : 33010 | 114번째 조회시작
114 번쨰데이터가없습니다.
location : 33010 | 115번째 조회시작
location : 33010 | 116번

location : 33030 | 91번째 조회시작
location : 33030 | 92번째 조회시작
location : 33030 | 93번째 조회시작
location : 33030 | 94번째 조회시작
location : 33030 | 95번째 조회시작
location : 33030 | 96번째 조회시작
location : 33030 | 97번째 조회시작
location : 33030 | 98번째 조회시작
location : 33030 | 99번째 조회시작
99 번쨰데이터가없습니다.
location : 33030 | 100번째 조회시작
100 번쨰데이터가없습니다.
location : 33030 | 101번째 조회시작
101 번쨰데이터가없습니다.
location : 33030 | 102번째 조회시작
102 번쨰데이터가없습니다.
location : 33030 | 103번째 조회시작
103 번쨰데이터가없습니다.
location : 33030 | 104번째 조회시작
104 번쨰데이터가없습니다.
location : 33030 | 105번째 조회시작
105 번쨰데이터가없습니다.
location : 33030 | 106번째 조회시작
106 번쨰데이터가없습니다.
location : 33030 | 107번째 조회시작
107 번쨰데이터가없습니다.
location : 33030 | 108번째 조회시작
108 번쨰데이터가없습니다.
location : 33030 | 109번째 조회시작
109 번쨰데이터가없습니다.
location : 33030 | 110번째 조회시작
110 번쨰데이터가없습니다.
location : 33030 | 111번째 조회시작
111 번쨰데이터가없습니다.
location : 33030 | 112번째 조회시작
112 번쨰데이터가없습니다.
location : 33030 | 113번째 조회시작
113 번쨰데이터가없습니다.
location : 33030 | 114번째 조회시작
114 번쨰데이터가없습니다.
location : 33030 | 115번째 조회시작
115 

80 번쨰데이터가없습니다.
location : 33330 | 81번째 조회시작
81 번쨰데이터가없습니다.
location : 33330 | 82번째 조회시작
82 번쨰데이터가없습니다.
location : 33330 | 83번째 조회시작
83 번쨰데이터가없습니다.
location : 33330 | 84번째 조회시작
84 번쨰데이터가없습니다.
location : 33330 | 85번째 조회시작
85 번쨰데이터가없습니다.
location : 33330 | 86번째 조회시작
86 번쨰데이터가없습니다.
location : 33330 | 87번째 조회시작
87 번쨰데이터가없습니다.
location : 33330 | 88번째 조회시작
88 번쨰데이터가없습니다.
location : 33330 | 89번째 조회시작
89 번쨰데이터가없습니다.
location : 33330 | 90번째 조회시작
90 번쨰데이터가없습니다.
location : 33330 | 91번째 조회시작
91 번쨰데이터가없습니다.
location : 33330 | 92번째 조회시작
92 번쨰데이터가없습니다.
location : 33330 | 93번째 조회시작
93 번쨰데이터가없습니다.
location : 33330 | 94번째 조회시작
94 번쨰데이터가없습니다.
location : 33330 | 95번째 조회시작
95 번쨰데이터가없습니다.
location : 33330 | 96번째 조회시작
96 번쨰데이터가없습니다.
location : 33330 | 97번째 조회시작
97 번쨰데이터가없습니다.
location : 33330 | 98번째 조회시작
98 번쨰데이터가없습니다.
location : 33330 | 99번째 조회시작
99 번쨰데이터가없습니다.
location : 33330 | 100번째 조회시작
100 번쨰데이터가없습니다.
location : 33330 | 101번째 조회시작
101 번쨰데이터가없습니다.
location : 33330 | 102번째 조회시작
102 번쨰데이터가없습니다.
location : 

location : 33350 | 54번째 조회시작
54 번쨰데이터가없습니다.
location : 33350 | 55번째 조회시작
location : 33350 | 56번째 조회시작
56 번쨰데이터가없습니다.
location : 33350 | 57번째 조회시작
57 번쨰데이터가없습니다.
location : 33350 | 58번째 조회시작
58 번쨰데이터가없습니다.
location : 33350 | 59번째 조회시작
59 번쨰데이터가없습니다.
location : 33350 | 60번째 조회시작
location : 33350 | 61번째 조회시작
61 번쨰데이터가없습니다.
location : 33350 | 62번째 조회시작
62 번쨰데이터가없습니다.
location : 33350 | 63번째 조회시작
63 번쨰데이터가없습니다.
location : 33350 | 64번째 조회시작
64 번쨰데이터가없습니다.
location : 33350 | 65번째 조회시작
65 번쨰데이터가없습니다.
location : 33350 | 66번째 조회시작
66 번쨰데이터가없습니다.
location : 33350 | 67번째 조회시작
location : 33350 | 68번째 조회시작
68 번쨰데이터가없습니다.
location : 33350 | 69번째 조회시작
69 번쨰데이터가없습니다.
location : 33350 | 70번째 조회시작
location : 33350 | 71번째 조회시작
location : 33350 | 72번째 조회시작
72 번쨰데이터가없습니다.
location : 33350 | 73번째 조회시작
73 번쨰데이터가없습니다.
location : 33350 | 74번째 조회시작
location : 33350 | 75번째 조회시작
75 번쨰데이터가없습니다.
location : 33350 | 76번째 조회시작
location : 33350 | 77번째 조회시작
77 번쨰데이터가없습니다.
location : 33350 | 78번째 조회시작
location : 33350 | 7